# LSTM models on the UCI datasets
- LSTM Models are slow to train. The newly proposed TCNs are shown to be simpler and easier to train, along with better performance

In [1]:
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from matplotlib import pyplot

In [3]:
from tensorflow.keras.utils import to_categorical

In [ ]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess= tf.Session(config=config):

# Data loading utilities

In [4]:
def load_file(filepath):
    dataframe = read_csv(filepath, header=None, delim_whitespace=True)
    return dataframe.values
 
def load_group(filenames, prefix=''):
    loaded = list()
    for name in filenames:
        data = load_file(prefix + name)
        #print(data.shape)
        loaded.append(data)
        
    loaded = dstack(loaded)
    return loaded

# load train or test
def load_dataset_group(group, prefix=''):
    filepath = prefix + group + '/Inertial Signals/'
    # load all 9 files as a single array
    filenames = list()
    # total acceleration
    filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
    # body acceleration
    filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
    # body gyroscope
    filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
    # load input data
    X = load_group(filenames, filepath)
    # load class output
    y = load_file(prefix + group + '/y_'+group+'.txt')
    return X, y


# load the dataset, returns train and test X and y elements
def load_dataset(prefix=''):
    # load all train
    trainX, trainy = load_dataset_group('train', prefix + 'datasets/UCI-HAR-Dataset/')
    
    # load all test
    testX, testy = load_dataset_group('test', prefix + 'datasets/UCI-HAR-Dataset/')
    
    # zero-offset class values
    trainy = trainy - 1
    testy = testy - 1
    # one hot encode y
    trainy = to_categorical(trainy)
    testy = to_categorical(testy)
    print(trainX.shape, trainy.shape, testX.shape, testy.shape)
    return trainX, trainy, testX, testy

In [5]:
# Load the data
trainX, trainy, testX, testy = load_dataset()

(7352, 128, 9) (7352, 6) (2947, 128, 9) (2947, 6)


# Now let us create a MLP network first

In [6]:
# MLP with take 1-D data as an input

# Load the data
trainX, trainy, testX, testy = load_dataset()

trainX = trainX.reshape(7352,-1)
print(trainX.shape)

testX = testX.reshape(2947, -1)
print(testX.shape)

(7352, 128, 9) (7352, 6) (2947, 128, 9) (2947, 6)
(7352, 1152)
(2947, 1152)


In [8]:
# MLP Network
import tensorflow as tf

model = tf.keras.models.Sequential()
model.add(tf.keras.Input(shape=(trainX.shape[1],)))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(trainy.shape[1], activation='softmax'))
#model.output_shape

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 64)                73792     
_________________________________________________________________
dense_5 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_6 (Dense)              (None, 32)                2080      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 6)                 198       
Total params: 80,230
Trainable params: 80,230
Non-trainable params: 0
_________________________________________________________________


In [11]:
filepath = 'MLP_Best.hdf5'
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=filepath, 
                             monitor='val_accuracy',
                             verbose=1, 
                             save_best_only=True,
                             mode='max')

callbacks = [checkpoint]

model.fit(trainX, trainy, batch_size=64, epochs=100,
            verbose=0, validation_data=(testX, testy),
         callbacks = callbacks)


Epoch 00001: val_accuracy improved from -inf to 0.89006, saving model to MLP_Best.hdf5

Epoch 00002: val_accuracy did not improve from 0.89006

Epoch 00003: val_accuracy did not improve from 0.89006

Epoch 00004: val_accuracy did not improve from 0.89006

Epoch 00005: val_accuracy did not improve from 0.89006

Epoch 00006: val_accuracy did not improve from 0.89006

Epoch 00007: val_accuracy did not improve from 0.89006

Epoch 00008: val_accuracy did not improve from 0.89006

Epoch 00009: val_accuracy did not improve from 0.89006

Epoch 00010: val_accuracy did not improve from 0.89006

Epoch 00011: val_accuracy did not improve from 0.89006

Epoch 00012: val_accuracy did not improve from 0.89006

Epoch 00013: val_accuracy did not improve from 0.89006

Epoch 00014: val_accuracy did not improve from 0.89006

Epoch 00015: val_accuracy did not improve from 0.89006

Epoch 00016: val_accuracy did not improve from 0.89006

Epoch 00017: val_accuracy did not improve from 0.89006

Epoch 00018: va

In [12]:
model.load_weights(filepath)

# Re-evaluate the model
loss, acc = model.evaluate(testX, testy, verbose=2)
print("MLP model, accuracy: {:5.2f}%".format(100 * acc))

93/93 - 0s - loss: 0.9975 - accuracy: 0.8901
MLP model, accuracy: 89.01%


# Let us do a 1-D CNN

In [13]:
# Load the data
trainX, trainy, testX, testy = load_dataset()

(7352, 128, 9) (7352, 6) (2947, 128, 9) (2947, 6)


In [14]:
# 1-D CNN Network
import tensorflow as tf

model = tf.keras.models.Sequential()
model.add(tf.keras.Input(shape=(trainX.shape[1], trainX.shape[2])))
model.add(tf.keras.layers.Conv1D(filters=64, kernel_size=5, activation='relu'))
model.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3, strides=2, activation='relu'))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(trainy.shape[1], activation='softmax'))
#model.output_shape

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 124, 64)           2944      
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 61, 32)            6176      
_________________________________________________________________
flatten (Flatten)            (None, 1952)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 32)                62496     
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 6)                 198       
Total params: 71,814
Trainable params: 71,814
Non-trainable params: 0
__________________________________________________

In [16]:
filepath = '1D_CNN_Best.hdf5'
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=filepath, 
                             monitor='val_accuracy',
                             verbose=1, 
                             save_best_only=True,
                             mode='max')

callbacks = [checkpoint]

model.fit(trainX, trainy, batch_size=64, epochs=100,
            verbose=1, validation_data=(testX, testy),
         callbacks = callbacks)

Epoch 1/100


2022-03-05 17:12:40.080132: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8
2022-03-05 17:12:43.278101: W tensorflow/stream_executor/gpu/asm_compiler.cc:63] Running ptxas --version returned 256
2022-03-05 17:12:43.355426: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: ptxas exited with non-zero error code 256, output: 
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


115/115 [==============================] - 11s 41ms/step - loss: 1.0306 - accuracy: 0.5777 - val_loss: 0.4357 - val_accuracy: 0.8534

Epoch 00001: val_accuracy improved from -inf to 0.85341, saving model to 1D_CNN_Best.hdf5
Epoch 2/100
115/115 [==============================] - 1s 5ms/step - loss: 0.3281 - accuracy: 0.8765 - val_loss: 0.2720 - val_accuracy: 0.9091

Epoch 00002: val_accuracy improved from 0.85341 to 0.90906, saving model to 1D_CNN_Best.hdf5
Epoch 3/100
115/115 [==============================] - 1s 5ms/step - loss: 0.1895 - accuracy: 0.9284 - val_loss: 0.2511 - val_accuracy: 0.9108

Epoch 00003: val_accuracy improved from 0.90906 to 0.91076, saving model to 1D_CNN_Best.hdf5
Epoch 4/100
115/115 [==============================] - 1s 5ms/step - loss: 0.1524 - accuracy: 0.9438 - val_loss: 0.2478 - val_accuracy: 0.9192

Epoch 00004: val_accuracy improved from 0.91076 to 0.91924, saving model to 1D_CNN_Best.hdf5
Epoch 5/100
115/115 [==============================] - 1s 5ms/ste

115/115 [==============================] - 1s 5ms/step - loss: 0.0674 - accuracy: 0.9631 - val_loss: 0.7040 - val_accuracy: 0.9101

Epoch 00041: val_accuracy did not improve from 0.93383
Epoch 42/100
115/115 [==============================] - 1s 5ms/step - loss: 0.0665 - accuracy: 0.9681 - val_loss: 0.6289 - val_accuracy: 0.9121

Epoch 00042: val_accuracy did not improve from 0.93383
Epoch 43/100
115/115 [==============================] - 1s 5ms/step - loss: 0.0701 - accuracy: 0.9633 - val_loss: 0.6414 - val_accuracy: 0.9162

Epoch 00043: val_accuracy did not improve from 0.93383
Epoch 44/100
115/115 [==============================] - 1s 5ms/step - loss: 0.0675 - accuracy: 0.9622 - val_loss: 0.6740 - val_accuracy: 0.9179

Epoch 00044: val_accuracy did not improve from 0.93383
Epoch 45/100
115/115 [==============================] - 1s 5ms/step - loss: 0.0673 - accuracy: 0.9657 - val_loss: 0.6350 - val_accuracy: 0.9237

Epoch 00045: val_accuracy did not improve from 0.93383
Epoch 46/100


115/115 [==============================] - 1s 5ms/step - loss: 0.0523 - accuracy: 0.9720 - val_loss: 0.9726 - val_accuracy: 0.9172

Epoch 00082: val_accuracy did not improve from 0.93383
Epoch 83/100
115/115 [==============================] - 1s 5ms/step - loss: 0.0484 - accuracy: 0.9748 - val_loss: 1.0054 - val_accuracy: 0.9152

Epoch 00083: val_accuracy did not improve from 0.93383
Epoch 84/100
115/115 [==============================] - 1s 5ms/step - loss: 0.0401 - accuracy: 0.9800 - val_loss: 1.0843 - val_accuracy: 0.9186

Epoch 00084: val_accuracy did not improve from 0.93383
Epoch 85/100
115/115 [==============================] - 1s 5ms/step - loss: 0.0458 - accuracy: 0.9782 - val_loss: 1.1424 - val_accuracy: 0.9172

Epoch 00085: val_accuracy did not improve from 0.93383
Epoch 86/100
115/115 [==============================] - 1s 5ms/step - loss: 0.0453 - accuracy: 0.9759 - val_loss: 1.1381 - val_accuracy: 0.9091

Epoch 00086: val_accuracy did not improve from 0.93383
Epoch 87/100


In [17]:
# Re-evaluate the model
model.load_weights(filepath)

loss, acc = model.evaluate(testX, testy, verbose=2)
print("1D CNN model, accuracy: {:5.2f}%".format(100 * acc))

93/93 - 1s - loss: 0.3812 - accuracy: 0.9338
1D CNN model, accuracy: 93.38%


# 2D Convolutions

In [18]:
trainX, trainy, testX, testy = load_dataset()

trainX = trainX.reshape(7352,trainX.shape[1], trainX.shape[2], 1)
print(trainX.shape)

testX = testX.reshape(2947, testX.shape[1], testX.shape[2], 1)
print(testX.shape)

(7352, 128, 9) (7352, 6) (2947, 128, 9) (2947, 6)
(7352, 128, 9, 1)
(2947, 128, 9, 1)


In [19]:
# 2-D CNN Network
import tensorflow as tf

model = tf.keras.models.Sequential()
model.add(tf.keras.Input(shape=(trainX.shape[1], trainX.shape[2], 1)))
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=5, activation='relu'))
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, strides=2, activation='relu'))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(trainy.shape[1], activation='softmax'))
#model.output_shape

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 124, 5, 64)        1664      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 61, 2, 32)         18464     
_________________________________________________________________
flatten_1 (Flatten)          (None, 3904)              0         
_________________________________________________________________
dense_10 (Dense)             (None, 32)                124960    
_________________________________________________________________
dropout_3 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 6)                 198       
Total params: 145,286
Trainable params: 145,286
Non-trainable params: 0
________________________________________________

In [21]:
filepath = '2D_CNN_Best.hdf5'
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=filepath, 
                             monitor='val_accuracy',
                             verbose=1, 
                             save_best_only=True,
                             mode='max')

callbacks = [checkpoint]

model.fit(trainX, trainy, batch_size=64, epochs=100,
            verbose=1, validation_data=(testX, testy),
         callbacks = callbacks)

Epoch 1/100
115/115 [==============================] - 9s 50ms/step - loss: 0.9626 - accuracy: 0.6378 - val_loss: 0.5160 - val_accuracy: 0.8042

Epoch 00001: val_accuracy improved from -inf to 0.80421, saving model to 2D_CNN_Best.hdf5
Epoch 2/100
115/115 [==============================] - 1s 5ms/step - loss: 0.3470 - accuracy: 0.8668 - val_loss: 0.4378 - val_accuracy: 0.8487

Epoch 00002: val_accuracy improved from 0.80421 to 0.84866, saving model to 2D_CNN_Best.hdf5
Epoch 3/100
115/115 [==============================] - 1s 5ms/step - loss: 0.1897 - accuracy: 0.9318 - val_loss: 0.3707 - val_accuracy: 0.8955

Epoch 00003: val_accuracy improved from 0.84866 to 0.89549, saving model to 2D_CNN_Best.hdf5
Epoch 4/100
115/115 [==============================] - 1s 5ms/step - loss: 0.1518 - accuracy: 0.9450 - val_loss: 0.3598 - val_accuracy: 0.9050

Epoch 00004: val_accuracy improved from 0.89549 to 0.90499, saving model to 2D_CNN_Best.hdf5
Epoch 5/100
115/115 [==============================] -

115/115 [==============================] - 1s 5ms/step - loss: 0.0306 - accuracy: 0.9867 - val_loss: 0.8002 - val_accuracy: 0.9304

Epoch 00040: val_accuracy did not improve from 0.94028
Epoch 41/100
115/115 [==============================] - 1s 5ms/step - loss: 0.0348 - accuracy: 0.9841 - val_loss: 0.8385 - val_accuracy: 0.9233

Epoch 00041: val_accuracy did not improve from 0.94028
Epoch 42/100
115/115 [==============================] - 1s 5ms/step - loss: 0.0340 - accuracy: 0.9871 - val_loss: 0.8288 - val_accuracy: 0.9267

Epoch 00042: val_accuracy did not improve from 0.94028
Epoch 43/100
115/115 [==============================] - 1s 5ms/step - loss: 0.0369 - accuracy: 0.9849 - val_loss: 0.9019 - val_accuracy: 0.9304

Epoch 00043: val_accuracy did not improve from 0.94028
Epoch 44/100
115/115 [==============================] - 1s 5ms/step - loss: 0.0339 - accuracy: 0.9849 - val_loss: 0.7680 - val_accuracy: 0.9315

Epoch 00044: val_accuracy did not improve from 0.94028
Epoch 45/100


115/115 [==============================] - 1s 6ms/step - loss: 0.0213 - accuracy: 0.9910 - val_loss: 2.3600 - val_accuracy: 0.9281

Epoch 00081: val_accuracy did not improve from 0.94028
Epoch 82/100
115/115 [==============================] - 1s 5ms/step - loss: 0.0174 - accuracy: 0.9933 - val_loss: 1.9623 - val_accuracy: 0.9287

Epoch 00082: val_accuracy did not improve from 0.94028
Epoch 83/100
115/115 [==============================] - 1s 5ms/step - loss: 0.0479 - accuracy: 0.9862 - val_loss: 1.8041 - val_accuracy: 0.9220

Epoch 00083: val_accuracy did not improve from 0.94028
Epoch 84/100
115/115 [==============================] - 1s 5ms/step - loss: 0.0162 - accuracy: 0.9938 - val_loss: 2.0896 - val_accuracy: 0.9247

Epoch 00084: val_accuracy did not improve from 0.94028
Epoch 85/100
115/115 [==============================] - 1s 5ms/step - loss: 0.0128 - accuracy: 0.9960 - val_loss: 2.2209 - val_accuracy: 0.9264

Epoch 00085: val_accuracy did not improve from 0.94028
Epoch 86/100


In [22]:
model.load_weights(filepath)
loss, acc = model.evaluate(testX, testy, verbose=2)
print("2D CNN model, accuracy: {:5.2f}%".format(100 * acc))

93/93 - 2s - loss: 0.4439 - accuracy: 0.9403
2D CNN model, accuracy: 94.03%


# Using only ACC and GYRO

In [23]:
def load_file(filepath):
    dataframe = read_csv(filepath, header=None, delim_whitespace=True)
    return dataframe.values
 
def load_group(filenames, prefix=''):
    loaded = list()
    for name in filenames:
        data = load_file(prefix + name)
        #print(data.shape)
        loaded.append(data)
        
    loaded = dstack(loaded)
    return loaded

# load train or test
def load_dataset_group(group, prefix=''):
    filepath = prefix + group + '/Inertial Signals/'
    # load all 9 files as a single array
    filenames = list()
    # body acceleration
    filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
    # body gyroscope
    filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
    # load input data
    X = load_group(filenames, filepath)
    # load class output
    y = load_file(prefix + group + '/y_'+group+'.txt')
    return X, y


# load the dataset, returns train and test X and y elements
def load_dataset(prefix=''):
    # load all train
    trainX, trainy = load_dataset_group('train', prefix + 'datasets/UCI-HAR-Dataset/')
    
    # load all test
    testX, testy = load_dataset_group('test', prefix + 'datasets/UCI-HAR-Dataset/')
    
    # zero-offset class values
    trainy = trainy - 1
    testy = testy - 1
    # one hot encode y
    trainy = to_categorical(trainy)
    testy = to_categorical(testy)
    print(trainX.shape, trainy.shape, testX.shape, testy.shape)
    return trainX, trainy, testX, testy

In [24]:
# Load the data
trainX, trainy, testX, testy = load_dataset()

(7352, 128, 6) (7352, 6) (2947, 128, 6) (2947, 6)


In [25]:
# 1-D CNN Network
import tensorflow as tf

model = tf.keras.models.Sequential()
model.add(tf.keras.Input(shape=(trainX.shape[1], trainX.shape[2])))
model.add(tf.keras.layers.Conv1D(filters=64, kernel_size=5, activation='relu'))
model.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3, strides=2, activation='relu'))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(trainy.shape[1], activation='softmax'))
#model.output_shape

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_2 (Conv1D)            (None, 124, 64)           1984      
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 61, 32)            6176      
_________________________________________________________________
flatten_2 (Flatten)          (None, 1952)              0         
_________________________________________________________________
dense_12 (Dense)             (None, 32)                62496     
_________________________________________________________________
dropout_4 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_13 (Dense)             (None, 6)                 198       
Total params: 70,854
Trainable params: 70,854
Non-trainable params: 0
__________________________________________________

In [27]:
filepath = '1D_CNN_Best_2.hdf5'
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=filepath, 
                             monitor='val_accuracy',
                             verbose=1, 
                             save_best_only=True,
                             mode='max')

callbacks = [checkpoint]

model.fit(trainX, trainy, batch_size=64, epochs=100,
            verbose=1, validation_data=(testX, testy),
         callbacks = callbacks)

Epoch 1/100
115/115 [==============================] - 5s 26ms/step - loss: 1.4398 - accuracy: 0.3380 - val_loss: 0.7383 - val_accuracy: 0.6240

Epoch 00001: val_accuracy improved from -inf to 0.62402, saving model to 1D_CNN_Best_2.hdf5
Epoch 2/100
115/115 [==============================] - 1s 4ms/step - loss: 0.7244 - accuracy: 0.6284 - val_loss: 0.6601 - val_accuracy: 0.6960

Epoch 00002: val_accuracy improved from 0.62402 to 0.69596, saving model to 1D_CNN_Best_2.hdf5
Epoch 3/100
115/115 [==============================] - 1s 5ms/step - loss: 0.6273 - accuracy: 0.6694 - val_loss: 0.7005 - val_accuracy: 0.6471

Epoch 00003: val_accuracy did not improve from 0.69596
Epoch 4/100
115/115 [==============================] - 1s 5ms/step - loss: 0.6044 - accuracy: 0.6787 - val_loss: 0.6320 - val_accuracy: 0.6970

Epoch 00004: val_accuracy improved from 0.69596 to 0.69698, saving model to 1D_CNN_Best_2.hdf5
Epoch 5/100
115/115 [==============================] - 1s 5ms/step - loss: 0.5663 - ac

115/115 [==============================] - 1s 5ms/step - loss: 0.1396 - accuracy: 0.9483 - val_loss: 0.5155 - val_accuracy: 0.9009

Epoch 00038: val_accuracy did not improve from 0.90702
Epoch 39/100
115/115 [==============================] - 1s 5ms/step - loss: 0.1454 - accuracy: 0.9468 - val_loss: 0.5818 - val_accuracy: 0.8972

Epoch 00039: val_accuracy did not improve from 0.90702
Epoch 40/100
115/115 [==============================] - 1s 5ms/step - loss: 0.1388 - accuracy: 0.9489 - val_loss: 0.5532 - val_accuracy: 0.8965

Epoch 00040: val_accuracy did not improve from 0.90702
Epoch 41/100
115/115 [==============================] - 1s 5ms/step - loss: 0.1370 - accuracy: 0.9477 - val_loss: 0.5410 - val_accuracy: 0.8897

Epoch 00041: val_accuracy did not improve from 0.90702
Epoch 42/100
115/115 [==============================] - 1s 5ms/step - loss: 0.1233 - accuracy: 0.9568 - val_loss: 0.5315 - val_accuracy: 0.8921

Epoch 00042: val_accuracy did not improve from 0.90702
Epoch 43/100


115/115 [==============================] - 1s 5ms/step - loss: 0.0651 - accuracy: 0.9775 - val_loss: 0.8902 - val_accuracy: 0.9036

Epoch 00078: val_accuracy did not improve from 0.91449
Epoch 79/100
115/115 [==============================] - 1s 5ms/step - loss: 0.1050 - accuracy: 0.9678 - val_loss: 0.6354 - val_accuracy: 0.9074

Epoch 00079: val_accuracy did not improve from 0.91449
Epoch 80/100
115/115 [==============================] - 1s 5ms/step - loss: 0.0632 - accuracy: 0.9765 - val_loss: 0.6272 - val_accuracy: 0.9121

Epoch 00080: val_accuracy did not improve from 0.91449
Epoch 81/100
115/115 [==============================] - 1s 5ms/step - loss: 0.0576 - accuracy: 0.9790 - val_loss: 0.6758 - val_accuracy: 0.9121

Epoch 00081: val_accuracy did not improve from 0.91449
Epoch 82/100
115/115 [==============================] - 1s 5ms/step - loss: 0.0584 - accuracy: 0.9808 - val_loss: 0.6811 - val_accuracy: 0.9141

Epoch 00082: val_accuracy did not improve from 0.91449
Epoch 83/100


In [28]:
# Re-evaluate the model
model.load_weights(filepath)

loss, acc = model.evaluate(testX, testy, verbose=2)
print("1D CNN model, accuracy: {:5.2f}%".format(100 * acc))

93/93 - 1s - loss: 0.7504 - accuracy: 0.9162
1D CNN model, accuracy: 91.62%


# Using LSTM

In [29]:
def load_file(filepath):
    dataframe = read_csv(filepath, header=None, delim_whitespace=True)
    return dataframe.values
 
def load_group(filenames, prefix=''):
    loaded = list()
    for name in filenames:
        data = load_file(prefix + name)
        #print(data.shape)
        loaded.append(data)
        
    loaded = dstack(loaded)
    return loaded

# load train or test
def load_dataset_group(group, prefix=''):
    filepath = prefix + group + '/Inertial Signals/'
    # load all 9 files as a single array
    filenames = list()
    # total acceleration
    filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
    # body acceleration
    filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
    # body gyroscope
    filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
    # load input data
    X = load_group(filenames, filepath)
    # load class output
    y = load_file(prefix + group + '/y_'+group+'.txt')
    return X, y


# load the dataset, returns train and test X and y elements
def load_dataset(prefix=''):
    # load all train
    trainX, trainy = load_dataset_group('train', prefix + 'datasets/UCI-HAR-Dataset/')
    
    # load all test
    testX, testy = load_dataset_group('test', prefix + 'datasets/UCI-HAR-Dataset/')
    
    # zero-offset class values
    trainy = trainy - 1
    testy = testy - 1
    # one hot encode y
    trainy = to_categorical(trainy)
    testy = to_categorical(testy)
    print(trainX.shape, trainy.shape, testX.shape, testy.shape)
    return trainX, trainy, testX, testy



In [30]:
trainX, trainy, testX, testy = load_dataset()

(7352, 128, 9) (7352, 6) (2947, 128, 9) (2947, 6)


In [39]:
# LSTM Network Default input: [batch, timesteps, feature]
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess= tf.Session(config=config)


model = tf.keras.models.Sequential()
model.add(tf.keras.Input(shape=(trainX.shape[1], trainX.shape[2])))
model.add(tf.keras.layers.LSTM(16, activation='relu'))
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(trainy.shape[1], activation='softmax'))
#model.output_shape

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

model.summary()

2022-03-05 17:27:42.902392: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-03-05 17:27:42.902975: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.683GHz coreCount: 28 deviceMemorySize: 10.92GiB deviceMemoryBandwidth: 451.17GiB/s
2022-03-05 17:27:42.903985: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:02:00.0 name: NVIDIA GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.683GHz coreCount: 28 deviceMemorySize: 10.92GiB deviceMemoryBandwidth: 451.17GiB/s
2022-03-05 17:27:42.904070: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-03-05 17:27:42.904119: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libc

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 16)                1664      
_________________________________________________________________
dense_18 (Dense)             (None, 32)                544       
_________________________________________________________________
dropout_6 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_19 (Dense)             (None, 6)                 198       
Total params: 2,406
Trainable params: 2,406
Non-trainable params: 0
_________________________________________________________________


In [40]:
filepath = 'LSTM_Best.hdf5'
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=filepath, 
                             monitor='val_acc',
                             verbose=1, 
                             save_best_only=True,
                             mode='max')

callbacks = [checkpoint]

model.fit(trainX, trainy, batch_size=64, epochs=20,
            verbose=1, validation_data=(testX, testy),
         callbacks = callbacks)

Train on 7352 samples, validate on 2947 samples
Epoch 1/20
7352/7352 [==============================] - ETA: 0s - loss: 8.0648 - acc: 0.4316
Epoch 00001: val_acc improved from -inf to 0.50458, saving model to LSTM_Best.hdf5
7352/7352 [==============================] - 17s 2ms/sample - loss: 8.0648 - acc: 0.4316 - val_loss: 2.0300 - val_acc: 0.5046
Epoch 2/20
7352/7352 [==============================] - ETA: 0s - loss: 1.3778 - acc: 0.5362
Epoch 00002: val_acc improved from 0.50458 to 0.51612, saving model to LSTM_Best.hdf5
7352/7352 [==============================] - 16s 2ms/sample - loss: 1.3778 - acc: 0.5362 - val_loss: 1.5109 - val_acc: 0.5161
Epoch 3/20
7352/7352 [==============================] - ETA: 0s - loss: 1.2239 - acc: 0.5556
Epoch 00003: val_acc improved from 0.51612 to 0.53410, saving model to LSTM_Best.hdf5
7352/7352 [==============================] - 16s 2ms/sample - loss: 1.2239 - acc: 0.5556 - val_loss: 2.0348 - val_acc: 0.5341
Epoch 4/20
7352/7352 [==================

In [41]:
# Re-evaluate the model
model.load_weights(filepath)

loss, acc = model.evaluate(testX, testy, verbose=2)
print("LSTM model, accuracy: {:5.2f}%".format(100 * acc))

LSTM model, accuracy: 63.76%


# Using BI-Directional LSTM

In [43]:
# LSTM Network Default input: [batch, timesteps, feature]
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess= tf.Session(config=config)


model = tf.keras.models.Sequential()
model.add(tf.keras.Input(shape=(trainX.shape[1], trainX.shape[2])))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16, activation='relu')))
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(trainy.shape[1], activation='softmax'))
#model.output_shape

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

model.summary()

2022-03-05 17:34:25.290234: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-03-05 17:34:25.291441: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.683GHz coreCount: 28 deviceMemorySize: 10.92GiB deviceMemoryBandwidth: 451.17GiB/s
2022-03-05 17:34:25.292855: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:02:00.0 name: NVIDIA GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.683GHz coreCount: 28 deviceMemorySize: 10.92GiB deviceMemoryBandwidth: 451.17GiB/s
2022-03-05 17:34:25.292912: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-03-05 17:34:25.292942: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libc

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 32)                3328      
_________________________________________________________________
dense_22 (Dense)             (None, 32)                1056      
_________________________________________________________________
dropout_8 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_23 (Dense)             (None, 6)                 198       
Total params: 4,582
Trainable params: 4,582
Non-trainable params: 0
_________________________________________________________________


In [44]:
filepath = 'BiLSTM_Best.hdf5'
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=filepath, 
                             monitor='val_acc',
                             verbose=1, 
                             save_best_only=True,
                             mode='max')

callbacks = [checkpoint]

model.fit(trainX, trainy, batch_size=64, epochs=20,
            verbose=1, validation_data=(testX, testy),
         callbacks = callbacks)

Train on 7352 samples, validate on 2947 samples
Epoch 1/20
7352/7352 [==============================] - ETA: 0s - loss: 1.9268 - acc: 0.4606
Epoch 00001: val_acc improved from -inf to 0.55989, saving model to BiLSTM_Best.hdf5
7352/7352 [==============================] - 32s 4ms/sample - loss: 1.9268 - acc: 0.4606 - val_loss: 1.8444 - val_acc: 0.5599
Epoch 2/20
7352/7352 [==============================] - ETA: 0s - loss: 1.2398 - acc: 0.5958
Epoch 00002: val_acc did not improve from 0.55989
7352/7352 [==============================] - 31s 4ms/sample - loss: 1.2398 - acc: 0.5958 - val_loss: 1.2163 - val_acc: 0.5579
Epoch 3/20
7352/7352 [==============================] - ETA: 0s - loss: 1.0919 - acc: 0.6224
Epoch 00003: val_acc improved from 0.55989 to 0.59518, saving model to BiLSTM_Best.hdf5
7352/7352 [==============================] - 31s 4ms/sample - loss: 1.0919 - acc: 0.6224 - val_loss: 1.0664 - val_acc: 0.5952
Epoch 4/20
7352/7352 [==============================] - ETA: 0s - loss: 

In [45]:
# Re-evaluate the model
model.load_weights(filepath)

loss, acc = model.evaluate(testX, testy, verbose=2)
print("LSTM model, accuracy: {:5.2f}%".format(100 * acc))

LSTM model, accuracy: 66.98%
